# An Introduction to Natural Language Processing

The study of natural language processing is primarily concerned with teaching machines to process and "understand" text.  It is fairly straight forward for machines to understand structured data like numbers, however to understand language is far more difficult.  Part of this has to do with the nature of numbers versus the nature of language.  Specifically, words can have multiple meanings in multiple contexts.  While the mean behind a number can change, based on its reference, the definition of a number is consistent across all mediums and contexts.

To make this discrete, if you have 5 chairs or 5 starfish, while the type of thing being described changes, the quantity of things being described stays constant.  However, with language we cannot make such a claim.  For instance, let's look at the following sentence:

"I hope you do a great job!"

Taken out of context, its meaning is most likely that the narrator is expressing hope to the subject to do a great job.  Let's see the same sentence in another context:

Person1: "I am going to have the best project"

Person2: "Yea, right, you know I'll do better"

Person1: "I hope you do a great job."

Person1: "Not"

Person2: "No need to be sarcastic"

Here "I hope you do a great job" is intended _sarcastically_ so the meaning of the statement is reversed, despite us not changing the phrasing of the sentence, the _meaning_ has changed completely.  How do we even begin to encode something like sarcasm in our language?  Would a person even do that in a context we care about?  These are just some of the exciting questions of natural language processing!  

Below we'll make our way through the following topics:

* Building A Simple Recommendation Engine
    * Bag Of Words Model
    * stop words
    * stemming
    * lemmatization
    * part of speech tagging
        * n-gram analysis
    * named entity recognition
* scikit-learn for POS
* Relation Extraction
* syntax trees
* tf-idf
* corpus generation
* word2vec
* Topic modeling with LDA
* Text Classification with Naive Bayes
* Text Prediction with Logistic Regression

* Fuzzy Matching For Deduplication
* Finding Human Traffickers Online

References:
* https://moj-analytical-services.github.io/NLP-guidance/FeatureSelection.html
* https://towardsdatascience.com/feature-selection-on-text-classification-1b86879f548e
* https://machinelearningmastery.com/feature-selection-machine-learning-python/
* https://nlp.stanford.edu/IR-book/html/htmledition/contents-1.html
* http://blog.datumbox.com/using-feature-selection-methods-in-text-classification/
* https://www.nltk.org/book/
* https://spacy.io/usage/spacy-101/
* https://course.spacy.io/
* https://pythonspot.com/nltk-stop-words/
* https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html
* https://medium.com/@kangeugine/hidden-markov-model-7681c22f5b9
* https://www.nltk.org/book/ch05.html
* https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python
* https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da
* https://nlpforhackers.io/training-pos-tagger/
* https://nlpforhackers.io/named-entity-extraction/
* https://nlpforhackers.io/training-ner-large-dataset/

## The Bag Of Words Model

To get to a point where we can effectively model language, we need to consider it grounded in some task.  Here our task will always be the same, at a high level, how does a machine understand language?  You'll find the specific tasks that follow are mere consequences of this overarching theme of first, how do we understand language?  And as a secondary question, what aspects of language are useful for our specific understanding?  Therefore, any model, in principal will be interested in segmenting natural language into a decomposed or component state.

Let's begin with our first model and a specific sub-task, which will inform what's important.  Here we consider the bag of words model and the task of simple information retrieval.  

For this system we will need two parts:

* a system for transforming the text into numbers
* a system for doing the information retrival

Let's first define the text into numbers part of the system - 

## Enter the Bag Of Words Model

The Bag of words model is possibly the simplest model you could think of, let's see some code to implement it:

In [12]:
import string
import re

def get_bag_of_words(text):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = regex.sub('', text)
    tokens = [elem.rstrip() for elem in text.split(" ")]
    return {token:tokens.count(token) for token in tokens}

sentence = "Hello there friends, how are you?"
get_bag_of_words(sentence)

{'Hello': 1, 'there': 1, 'friends': 1, 'how': 1, 'are': 1, 'you': 1}

There are a couple of things to notice here:

1. we don't want to keep the punctuation
2. We now have a set of numbers that have lost semantic meaning

Now let's go about defining our simplicitic informaiton retrevial system.

Let's assume that we have a web application that should query something different depending on what a user types in.  We give them a "search" bar to look up information.  Let's assume for simplicity, that they can only type in keywords.

A good example for this is job search based on keyword.  Here, someone enters a role, like "data scientist" and open data science roles are returned.  How could we return the results?  Well the simplest way is with a look up table.  Now we are in a position to set up the second part of the system.  Let's see how that might get coded:

In [22]:
import string
import re

def get_bag_of_words(text, space_of_words):
    text = text.lower()
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = regex.sub('', text)
    tokens = [elem.rstrip() for elem in text.split(" ")]
    return [(word,tokens.count(word)) for word in space_of_words]

def jobs_to_return(job_phrase):
    if job_phrase[0][1] >= 1:
        if job_phrase[1][1] >= 1:
            return "Looking for a mid level data scientist"
        elif job_phrase[2][1] >= 1:
            return "Looking for a senior data engineer"
        elif job_phrase[3][1] >= 1:
            return "Looking for an experienced data manager"
    else:
        return "Sorry, we don't have any jobs"

space_of_words = "data scientist engineer manager".split()
sentence = "Data Scientist"
job_phrase = get_bag_of_words(sentence, space_of_words)
print(job_phrase)
jobs_to_return(job_phrase)

[('data', 1), ('scientist', 1), ('engineer', 0), ('manager', 0)]


'Looking for a mid level data scientist'

As you can see here this system is extremely simplistic, but it shows a possible design that could be implemented in the real world - If you have a web connection, you can check out a system I helped with called CALC that uses this very idea:

[https://calc.gsa.gov/](https://calc.gsa.gov/)

Check it out!

So far we've built an incredibly symplistic search engine.  One we could make it more natural, is by allowing for more flexible queries that help humans express what they are after, but that aren't important for our query.

## Enter Stop Words

First we'll see an example of how to do stop words on their own and then we'll add stop words to our system:

In [23]:
def remove_stop_words(sentence):
    stop_words = "hey i a i'm".split()
    return " ".join([word 
                     for word in sentence.split() 
                     if word not in stop_words])

sentence = "Hey Fred, I'm looking for a new car, do you have any recommendations?"
remove_stop_words(sentence)

"Hey Fred, I'm looking for new car, do you have any recommendations?"

The basic idea here is that we remove words that occur a lot in every day language, but don't hold semantically relevant information.  Most stop word lists are standard and come from analysis of major bodies of text called corpora or corpus in the singular.  From these corpora are massive bodies of text, that are supposed to capture the frequency of language in a general setting.  Of course, domain specific corpora exist as well.  For instance, words in the medical community are likely to have a different frequency and usage than in say the gaming community.  So we can differentiate dialetics and communities, in theory, from the language they use and directly from the frequency and occurence of different types of words.

Let's look at a standard set of stop words, from a very popular natural language processing library - Natural Language Toolkit (NLTK):

In [76]:
from nltk.corpus import stopwords
words = stopwords.words("english")
for index in range(10, len(words), 10):
    print(words[index-10:index])
    print()
print(len(stopwords.words("english")))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

["you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']

['himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself']

['they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this']

['that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be']

['been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing']

['a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until']

['while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into']

['through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down']

['in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once']

['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']

['few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only']

['own',

As you can see there are 179 words that occur commonly and don't convey meaning we care about for our information retrieval problem, or for some NLP tasks more generally.  Let's make use of the nltk stop words in or problem to see what we get out now:

In [26]:
from nltk.corpus import stopwords

def remove_stop_words(sentence):
    stop_words = stopwords.words("english")
    return " ".join([word 
                     for word in sentence.split() 
                     if word not in stop_words])

sentence = "Hey Fred, I'm looking for a new car, do you have any recommendations?"
remove_stop_words(sentence)

"Hey Fred, I'm looking new car, recommendations?"

As you can see, we get to the crux of what is being asked for and can now move onto more sophisticated processing.  Let's add the stop words component to our job query engine, so we can add more "natural" language querying.

In [38]:
import string
import re
from nltk.corpus import stopwords

def remove_stop_words(sentence):
    stop_words = stopwords.words("english")
    return " ".join([word 
                     for word in sentence.split() 
                     if word not in stop_words])

def get_bag_of_words(text, space_of_words):
    text = text.lower()
    text = remove_stop_words(text)
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = regex.sub('', text)
    tokens = [elem.rstrip() for elem in text.split(" ")]
    return [(word,tokens.count(word)) for word in space_of_words]
    
def jobs_to_return(job_phrase):
    results = []
    if job_phrase[0][1] >= 1:
        if job_phrase[1][1] >= 1:
            results.append("Looking for a mid level data scientist")
        if job_phrase[2][1] >= 1:
            results.append("Looking for a senior data engineer")
        if job_phrase[3][1] >= 1:
            results.append("Looking for an experienced data manager")
    else:
        results.append("Sorry, we don't have any jobs")
    return results

space_of_words = "data scientist engineer manager".split()
sentence = "I'm looking for a Data Scientist job or a Data Engineer job"
job_phrases = get_bag_of_words(sentence, space_of_words)
print(job_phrases)
for job in jobs_to_return(job_phrases):
    print(job)

[('data', 2), ('scientist', 1), ('engineer', 1), ('manager', 0)]
Looking for a mid level data scientist
Looking for a senior data engineer


As you can see we now have a more flexible search engine because of the preprocessing we've done so far.  However we can pretty easily break our engine if we change our query slightly:

In [39]:
space_of_words = "data scientist engineer manager".split()
sentence = "I'm looking for a Data Scientist job or a Data Engineering job"
job_phrases = get_bag_of_words(sentence, space_of_words)
print(job_phrases)
for job in jobs_to_return(job_phrases):
    print(job)

[('data', 2), ('scientist', 1), ('engineer', 0), ('manager', 0)]
Looking for a mid level data scientist


What happened?!  Well it turns out by changing "data engineer" to the more natural "data engineering", we lose our second search result.  In order to recover it let's introduce our next concept - stemming

## Enter Stemming

Stemming is the idea of taking the stem of a word.  So in this case, the stem of engineering is engineer.  Stemming will also handle the case of engineer versus engineers.  Basically extra pieces of grammatical syntax are removed.  This is sort of like a form of regularization for words.  Because we create a standard representation for related words that only have slight variance in meaning.

Let's look at how we might implement a stemmer:

In [43]:
def get_stem(word):
    if word.endswith("s"):
        return word[:-1]
    elif word.endswith("ing"):
        return word[:-3]
    else:
        return word

sentence = "I heard you have the hiccups, have you tried jumping up and down?"
" ".join([get_stem(word) for word in sentence.split(" ")])

'I heard you have the hiccups, have you tried jump up and down?'

As you can see, we are able to get the stem of the word jumping - in this case jump.  Let's look at another example where our simplistic stemmer fais: 

In [44]:
sentence = "Hey!  Are you going running  later?  I'd love to come with you."
" ".join([get_stem(word) for word in sentence.split(" ")])

"Hey!  Are you go runn  later?  I'd love to come with you."

As you can see, "runn" is not the stem of "running", so we need a quite sophisticated stemmer to handle all the cases, essentially writing down a lot of grammatical rules and edge cases.  Because that in it of itself would be a large enough project, we won't do that here.  Instead we will make use of an off the shelf stemmer, in this case again from NLTK:

In [48]:
from nltk.stem import SnowballStemmer

def get_stem(word):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(word)

plurals = ['running', 'caresses', 'flies', 'dies', 'mules', 'denied',
           'died', 'agreed', 'owned', 'humbled', 'sized',
           'meeting', 'stating', 'siezing', 'itemization',
           'sensational', 'traditional', 'reference', 'colonizer',
           'plotted']

singles = [get_stem(plural) for plural in plurals]
print(' '.join(singles))

run caress fli die mule deni die agre own humbl size meet state siez item sensat tradit refer colon plot


As you can see the "running" case is now well handled.  Other cases aren't handled perfectly but it's still decent.  Here's an example of our off the shelf stemmer performing well:

In [49]:
print(SnowballStemmer("english").stem("generously"))

generous


Be advised, not all stemmers are created equal, and we can probably always do better by covering more edge cases.  Here is an example of a classic stemmer that doesn't do as well in this case, but is generally pretty good:

In [50]:
from nltk.stem import PorterStemmer

print(PorterStemmer().stem("generously"))

gener


Notice, the goal of a good stemmer is that the stemmer gives us the singular case.  Sometimes this is easy, other times, not so much.  So there is always a trade off.  Of course, if we can standardize to some typical expectation for a stem and get the root meaning of the word in question, then it doesn't matter if we cover all the edge cases, of which there will always be an ever expanding list.

Let's incorporate our Snowball stemmer into our engine to increase the size of our query surface:

In [62]:
import string
import re
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

def get_stem(word):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(word)

def remove_stop_words(sentence):
    stop_words = stopwords.words("english")
    return " ".join([word 
                     for word in sentence.split() 
                     if word not in stop_words])

def get_bag_of_words(text, space_of_words):
    text = text.lower()
    text = remove_stop_words(text)
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = regex.sub('', text)
    tokens = [elem.rstrip() for elem in text.split(" ")]
    tokens = [get_stem(token) for token in tokens]
    return [(word,tokens.count(word)) for word in space_of_words]
    
def jobs_to_return(job_phrase):
    results = []
    if job_phrase[0][1] >= 1:
        if job_phrase[1][1] >= 1:
            results.append("Looking for a mid level data scientist")
        if job_phrase[2][1] >= 1:
            results.append("Looking for a senior data engineer")
        if job_phrase[3][1] >= 1:
            results.append("Looking for an experienced data manager")
    else:
        results.append("Sorry, we don't have any jobs")
    return results

def process_query(query):
    space_of_words = "data scientist engin manager".split()
    job_phrases = get_bag_of_words(query, space_of_words)
    print("Job phrases", job_phrases)
    for job in jobs_to_return(job_phrases):
        print(job)
        

sentence_one = "I'm looking for a Data Scientist job or a Data Engineer job"
print("results for sentence one:")
process_query(sentence_one)
print()
print("results for sentence two:")
sentence_two = "I'm looking for a Data Scientist job or a Data Engineering job"
process_query(sentence_one)

results for sentence one:
Job phrases [('data', 2), ('scientist', 1), ('engin', 1), ('manager', 0)]
Looking for a mid level data scientist
Looking for a senior data engineer

results for sentence two:
Job phrases [('data', 2), ('scientist', 1), ('engin', 1), ('manager', 0)]
Looking for a mid level data scientist
Looking for a senior data engineer


Notice that now both of our cases work now!  Of course, we had to change our recognized word for "engineer" to "engin" which is not ideal.  However, this does mean we can cover more cases.  With all things, there is a trade off between specificity and flexibility.  

In [63]:
print("results for sentence two:")
sentence_two = "I'm looking for a Data Scientist job or a Data Engine job"
process_query(sentence_one)

results for sentence two:
Job phrases [('data', 2), ('scientist', 1), ('engin', 1), ('manager', 0)]
Looking for a mid level data scientist
Looking for a senior data engineer


As you can see above, we possibly get a false positive.  Of course, I don't think "data engine job" is a thing, so the only way this comes up is a typo.  

What if we wanted even more flexibility in our query?  Well we can take our processing even further with a technique called lemmatization.  Lemmatization is similar to stemming in that, the central component of the word is preserved while all other pieces are disgarded.  The major difference between stemming and lemmatization is stemming is based on a rules engine, while lemmatization makes us of formal theory to find the root of the word.  The stanford nlp group even goes so far as to call lemmatization the "right way", while stemming is seen as "a crude rules engine".  

## Enter Lemmatization

Because lemmatization is sophisticated, we won't attempt implementation here, but instead simply make use of NLTK's solution by way of example first:

In [64]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

print(lemmatizer.lemmatize("cats", pos=''))
print(lemmatizer.lemmatize("cacti"))
print(lemmatizer.lemmatize("geese"))
print(lemmatizer.lemmatize("rocks"))
print(lemmatizer.lemmatize("python"))
print(lemmatizer.lemmatize("better", pos="a"))
print(lemmatizer.lemmatize("best", pos="a"))
print(lemmatizer.lemmatize("run"))
print(lemmatizer.lemmatize("run",'v'))

cat
cactus
goose
rock
python
good
best
run
run


As you can see we can even add a part of speech for increases flexability and to ensure a greater degree of correctness.  Let's go back to our example above and see how our lemmatizer does:

In [65]:
from nltk.stem import WordNetLemmatizer

def get_lemma(word):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(word)

plurals = ['running', 'caresses', 'flies', 'dies', 'mules', 'denied',
           'died', 'agreed', 'owned', 'humbled', 'sized',
           'meeting', 'stating', 'siezing', 'itemization',
           'sensational', 'traditional', 'reference', 'colonizer',
           'plotted']

singles = [get_lemma(plural) for plural in plurals]
print(' '.join(singles))

running caress fly dy mule denied died agreed owned humbled sized meeting stating siezing itemization sensational traditional reference colonizer plotted


As you can see, on some of the cases lemmatization doesn't work that well at all, but in other cases, lemmatizing does significantly better.  For example, fly is handled much better with the lemmatizer than the stemmer.  

The reason the lemmatizer appears to do not as well, is because it doesn't have the part of speech.  Note that in the first example, we hinted that we'll need part of speech in order to do a good job.  For this we will need to consider some part of speech tagging automatically, or be forced to do this manually ourselves (the horror!!!).

Before we move onto explaining part of speech tagging in general, let's look at a motivating example as to why our lemmatizer may be shy about getting the lemmas for our plurals above:

`They refuse to permit us to obtain the refuse permit.`

In the above sentence, refuse is used twice - once meaning to deny and the second time meaning trash.  The difference in definition is made apparent by the part of speech in use!  So for some other words, the plural maybe the same but without the added context, we can't be sure of the underlying meaning.

## Enter Part Of Speech Tagging

Part of speech tagging is a wide ranging and increadibly powerful tool.  It's invention is one of the great watershed moments in natural language processing.  With it, we have a basic model of the syntax of natural language and therefore can get a sense of the meaning, via the syntax of the sentence.  

Over the years, we've continued to see continual improvement in part of speech tagging because of its central nature to many NLP tasks.  Due to the number of ways we could do part of speech tagging we will only consider a very simple model - n-gram analysis.

## N-Gram Analysis

The idea behind n-grams are very simple, let's consider the following sentence and it's 1-grams, 2-grams and 3-grams.

Sentence: "I love dogs and cats, they are great!"

1-grams:

`[("I"), ("love"), ("dogs"), ("and"), ("cats"), ("they"), ("are"), ("great")]`

2-grams:

`[("I love"), ("love dogs"), ("dogs and"), ("and cats"), ("cats they"), ("they are"), ("are great")]`

3-grams:

`[("I love dogs"), ("love dogs and"), ("dogs and cats"), ("and cats they"), ("cats they are"), ("they are great")]`

As you can see from the above example an "n"-gram is a sliding window that considers sets of words based on the order of their appearance in some text.  The n in question defines how many words occur in each gram, aka the size of the sliding window.  Notice that a single word can appear in more than one gram.

Let's see how to code an ngram function in python:

In [6]:
import re
import string

def ngram(n, text):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = regex.sub('', text)
    tokens = text.split(" ")
    return list(zip(*[tokens[i:] for i in range(n)]))

text = "I love dogs and cats, they are great!"
print(ngram(1, text))
print(ngram(2, text))
print(ngram(3, text))

[('I',), ('love',), ('dogs',), ('and',), ('cats',), ('they',), ('are',), ('great',)]
[('I', 'love'), ('love', 'dogs'), ('dogs', 'and'), ('and', 'cats'), ('cats', 'they'), ('they', 'are'), ('are', 'great')]
[('I', 'love', 'dogs'), ('love', 'dogs', 'and'), ('dogs', 'and', 'cats'), ('and', 'cats', 'they'), ('cats', 'they', 'are'), ('they', 'are', 'great')]


As you can see we get the same thing from the algorithm as we did from the example above!  

So how does this help us with part of speech tagging?  Well we can build what's called a markov model - that is a model that decides what to do next based on the most recent piece of memory.  Specifically we'll use n-grams to make a simple markov model - one that only takes into account the most recent parts of speech.

Specifically, we'll take in the current word and the n-1 most recent word's parts of speech and use those as context to figure out what the most likely part of speech is for the current word.  

Next let's implement a simple markov model for unigrams:

In [22]:
from collections import defaultdict

class MarkovModel:
    def __init__(self):
        self.parts_of_speech = ['CC', 'CD', 'DT', 'EX', 'FW', 
                                'IN', 'JJ', 'JJR', 'JJS', 'LS', 
                                'MD', 'NN', 'NNS', 'NNP', 'NNPS', 
                                'PDT', 'POS', 'PRP', 'PRP$', 'RB', 
                                'RBR', 'RBS', 'RP', 'TO', 'UH', 
                                'VB', 'VBD', 'VBG', 'VBN', 'VBP', 
                                'VBZ', 'WDT', 'WP', 'WP$', 'WRB', 
                                "AT", "BER", "BEG", "CS", "BEZ"]
        self.part_of_speech_map = {pos:index 
                                   for index, pos in enumerate(self.parts_of_speech)}
        self.probabilities = None
        
    def fit(self, X, y):
        self.probabilities = defaultdict(list)
        for index, elem in enumerate(X):
            if elem not in self.probabilities:
                self.probabilities[elem] = [0] * len(self.parts_of_speech)
            pos_index = self.part_of_speech_map[y[index]]
            self.probabilities[elem][pos_index] += 1
    
    def _get_max_index(self, elem):
        return self.probabilities[elem].index(
            max(self.probabilities[elem])
        )
    
    def predict(self, X):
        pos = []
        for elem in X:
            index = self._get_max_index(elem)
            pos.append(
                self.parts_of_speech[index]
            )
        return pos

def ngram(n, text):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = regex.sub('', text)
    tokens = text.split(" ")
    return list(zip(*[tokens[i:] for i in range(n)]))

def accuracy(y_true, y_pred):
    count = 0
    for index in range(len(y_true)):
        if y_true[index] == y_pred[index]:
            count += 1
    return count / len(y_true)

mm = MarkovModel()
X = "Various of the apartments are of the terrace type, being on the ground floor so that entrace is direct."
y = ["JJ", "IN", "AT", "NNS", "BER", "IN", "AT", "NN", "NN", "BEG", "IN", "AT", "NN", "NN", "CS", "CS", "NN", "BEZ", "JJ"]
X = ngram(1, X)
mm.fit(X, y)
y_pred = mm.predict(X)
accuracy(y, y_pred)

1.0

of course, our model does really well because we tested on the same set we trained on.  Let's see the nltk equivalent at work on a real set of data:

In [23]:
from nltk.corpus import brown
import nltk
brown_tagged_sents = brown.tagged_sents(categories='news')
brown_sents = brown.sents(categories='news')
unigram_tagger = nltk.UnigramTagger(brown_tagged_sents)
for pair in unigram_tagger.tag(brown_sents[2007]):
    print(pair)
print(unigram_tagger.evaluate(brown_tagged_sents))

('Various', 'JJ')
('of', 'IN')
('the', 'AT')
('apartments', 'NNS')
('are', 'BER')
('of', 'IN')
('the', 'AT')
('terrace', 'NN')
('type', 'NN')
(',', ',')
('being', 'BEG')
('on', 'IN')
('the', 'AT')
('ground', 'NN')
('floor', 'NN')
('so', 'QL')
('that', 'CS')
('entrance', 'NN')
('is', 'BEZ')
('direct', 'JJ')
('.', '.')
0.9349006503968017


I leave it as an exercise to extend our model to the n-gram case.  But let's look at how easy it is to use Bigrams with nltk:

In [24]:
from nltk.corpus import brown
import nltk
brown_tagged_sents = brown.tagged_sents(categories='news')
brown_sents = brown.sents(categories='news')
bigram_tagger = nltk.BigramTagger(brown_tagged_sents)
for pair in bigram_tagger.tag(brown_sents[2007]):
    print(pair)
print(bigram_tagger.evaluate(brown_tagged_sents))

('Various', 'JJ')
('of', 'IN')
('the', 'AT')
('apartments', 'NNS')
('are', 'BER')
('of', 'IN')
('the', 'AT')
('terrace', 'NN')
('type', 'NN')
(',', ',')
('being', 'BEG')
('on', 'IN')
('the', 'AT')
('ground', 'NN')
('floor', 'NN')
('so', 'CS')
('that', 'CS')
('entrance', 'NN')
('is', 'BEZ')
('direct', 'JJ')
('.', '.')
0.7892972929967977


Interestingly, for this dataset using bigrams does not improve our models accuracy.  

## Adding Part of Speech Tagging to Lemmatization

Now that we know how to build a part of speech tagger, let's make use of the off the shelf one from nltk in order  to our lemmatizer.

In [7]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk

def get_pos_map(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''
    
def get_lemma(word):
    lemmatizer = WordNetLemmatizer()
    pos = nltk.pos_tag([word])[0][1]
    pos = get_pos_map(pos)
    return lemmatizer.lemmatize(word, pos=pos)

plurals = ['running', 'caresses', 'flies', 'dies', 'mules', 'denied',
           'died', 'agreed', 'owned', 'humbled', 'sized',
           'meeting', 'stating', 'siezing', 'itemization',
           'sensational', 'traditional', 'reference', 'colonizer',
           'plotted']

singles = [get_lemma(plural) for plural in plurals]
print(' '.join(singles))

run caress fly dy mule deny die agree own humble size meeting state siezing itemization sensational traditional reference colonizer plot


Woah!  Look at that improvement!  A lot more of the cases get covered now, even for this reasonably hard dataset. 

## Improving Our Information Retrieval System

Now let's expand our information retrieval task.  Let's include lemmatization and n-grams, so now we'll be able to parse phrases from our query!

In [21]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
import string
import re
from nltk.corpus import stopwords

def get_pos_map(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''
    
def get_lemma(word):
    lemmatizer = WordNetLemmatizer()
    pos = nltk.pos_tag([word])[0][1]
    pos = get_pos_map(pos)
    return lemmatizer.lemmatize(word, pos=pos)
    
def remove_stop_words(sentence):
    stop_words = stopwords.words("english")
    return " ".join([word 
                     for word in sentence.split() 
                     if word not in stop_words])

def get_bag_of_words(text, space_of_words):
    text = text.lower()
    text = remove_stop_words(text)
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = regex.sub('', text)
    tokens = [elem.rstrip() for elem in text.split(" ")]
    tokens = [get_lemma(token) for token in tokens]
    text = " ".join(tokens)
    tokens = ngram(1, text) + ngram(2, text) + ngram(3, text)
    tokens = post_process_grams(tokens)
    return [(word,tokens.count(word)) for word in space_of_words]
    
def ngram(n, text):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = regex.sub('', text)
    tokens = text.split(" ")
    return list(zip(*[tokens[i:] for i in range(n)]))

def post_process_grams(grams):
    return [" ".join(elem) for elem in grams]

def jobs_to_return(job_phrase):
    return ["We have a "+job[0]+" available" 
            for job in job_phrase if job[1] >= 1]

def process_query(query):
    space_of_words = ["data scientist", "data engineer", 
                      "machine learning engineer", "data science manager"
                     "data visualization engineer", "data engineering lead",
                     "junior data engineer", "senior data scientist"]
    job_phrases = get_bag_of_words(query, space_of_words)
    print("Job phrases", job_phrases)
    for job in jobs_to_return(job_phrases):
        print(job)
        

sentence_one = "I'm looking for a Senior Data Scientist job or a Junior Data Engineer job"
print("results for sentence one:")
process_query(sentence_one)
print()
print("results for sentence two:")
sentence_two = "I'm looking for a Data Scientist job or a Data Engineering job"
process_query(sentence_one)

results for sentence one:
Job phrases [('data scientist', 1), ('data engineer', 1), ('machine learning engineer', 0), ('data science managerdata visualization engineer', 0), ('data engineering lead', 0), ('junior data engineer', 1), ('senior data scientist', 1)]
We have a data scientist available
We have a data engineer available
We have a junior data engineer available
We have a senior data scientist available

results for sentence two:
Job phrases [('data scientist', 1), ('data engineer', 1), ('machine learning engineer', 0), ('data science managerdata visualization engineer', 0), ('data engineering lead', 0), ('junior data engineer', 1), ('senior data scientist', 1)]
We have a data scientist available
We have a data engineer available
We have a junior data engineer available
We have a senior data scientist available


We are now able to process more sophistcated queries such as whether the individual is senior or junior!  And now we don't need the kludgy thing we did before with "engin" instead of engineer or engineering!!

So far we've look at adding syntactic meaning to our data, next let's see how to add semantic meaning to our data with named entity recognition!

## Enter Named Entity Recognition

Named entity recognition is the process of adding meta data to proper nouns, identifying them by a category or possibly categories.  The process for training a named entity recognition system is very similar to training a part of speech tagger.  The features that go into the system are of course different as are the labels that come out but other than that the process is very similar.  

So let's go ahead and look at how nltk handles named entity recognition:

In [8]:
import nltk

def get_entities(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    sent = nltk.ne_chunk(sent)
    return sent

print(get_entities("I'm looking for a Senior Data Scientist job or a Junior Data Engineer job"))

(S
  I/PRP
  'm/VBP
  looking/VBG
  for/IN
  a/DT
  (ORGANIZATION Senior/NNP Data/NNP Scientist/NNP)
  job/NN
  or/CC
  a/DT
  (ORGANIZATION Junior/NNP Data/NNP Engineer/NNP)
  job/NN)


The NLTK parser thinks that the jobs are organizations, which isn't really accurate, but it doesn pick up that they are named entities!  Let's take a look how spacy, another nlp library does on this task, with the same sentence:

In [1]:
import spacy
# this can be downloaded from the command line with:
# python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
doc = nlp("I'm looking for a Senior Data Scientist job or a Junior Data Engineer job")
print([(token.text, token.label_) for token in doc.ents])

[('Junior Data Engineer', 'ORG')]


It looks like spacy doesn't do as well.  Let's try another case to see how the two fair:

In [10]:
print(get_entities('European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices'))

(S
  (GPE European/JJ)
  authorities/NNS
  fined/VBD
  (PERSON Google/NNP)
  a/DT
  record/NN
  $/$
  5.1/CD
  billion/CD
  on/IN
  Wednesday/NNP
  for/IN
  abusing/VBG
  its/PRP$
  power/NN
  in/IN
  the/DT
  mobile/JJ
  phone/NN
  market/NN
  and/CC
  ordered/VBD
  the/DT
  company/NN
  to/TO
  alter/VB
  its/PRP$
  practices/NNS)


Here NLTK gets one of the entities wrong - it thinks google is a person, but get's European correct - as it is a geopolitical organization!  Let's see how spacy fairs:

In [2]:
nlp = spacy.load("en_core_web_sm")
doc = nlp('European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices')
print([(X.text, X.label_) for X in doc.ents])

[('European', 'NORP'), ('Google', 'ORG'), ('$5.1 billion', 'MONEY'), ('Wednesday', 'DATE')]


On this case, spacy does way better!  It correctly identifies all the entities!  Even picking up some of the ones nltk missed!

## Training your own Part Of Speech Tagger

So far we've implemented our own algorithms for these NLP tasks.  But what if we wanted to use an off the shelf model from scikit-learn?

Well that's what we are going to see how to do now!  There won't be a ton of theory here, but we'll get to see how we can possibly improve performance over what people have done implemented in the major libraries.  This will also become incredibly important when we get to the active learning section.

In [10]:
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from functools import partial

def get_features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }
 
def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]
  
def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            X.append(get_features(untag(tagged), index))
            y.append(tagged[index][1])
 
    return X, y

def part_of_speech_tagger(clf, sentence):
    tags = clf.predict([get_features(sentence, index) for index in range(len(sentence))])
    return list(zip(sentence, tags))

tagged_sentences = nltk.corpus.treebank.tagged_sents()
data = tagged_sentences[:1000]
X, y = transform_to_dataset(data)
X_train, X_test, y_train, y_test = train_test_split(X, y)

clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy'))
])
 
clf.fit(X_train, y_train)   
y_pred = clf.predict(X_test)
pos_tag = partial(part_of_speech_tagger, clf)
print(pos_tag(nltk.word_tokenize('This is my friend, John.')))
print(classification_report(y_test, y_pred))

[('This', 'DT'), ('is', 'VBZ'), ('my', 'PRP$'), ('friend', 'NN'), (',', ','), ('John', 'NNP'), ('.', '.')]
              precision    recall  f1-score   support

           $       1.00      1.00      1.00        25
          ''       1.00      1.00      1.00        42
           ,       1.00      1.00      1.00       271
       -LRB-       1.00      1.00      1.00         4
      -NONE-       1.00      1.00      1.00       427
       -RRB-       1.00      1.00      1.00         4
           .       1.00      1.00      1.00       248
           :       1.00      1.00      1.00        33
          CC       0.99      0.97      0.98       151
          CD       0.99      0.98      0.99       173
          DT       0.99      0.98      0.98       527
          EX       1.00      1.00      1.00         7
          IN       0.97      0.93      0.95       627
          JJ       0.77      0.80      0.78       350
         JJR       0.79      0.65      0.71        17
         JJS       1.00     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


As you can see the crux of the work comes from having an already tagged dataset and doing the right feature engineering.  Then is is just a multiclass classification problem!  Even a somewhat naive model does quiet well as we can see from the classification report.  It does seem like some of the support for some of our classes is quiet small, so we might need to be careful with how much we trust our predictions overall, but for a worked example, this is pretty good!

We could do the same thing for Named Entity recognition as this [blog post](https://nlpforhackers.io/named-entity-extraction/) outlines, however downloading the dataset can be a pain, so we won't show the full code here, but we will look at the get_features function:

In [ ]:
def get_features(tokens, index, history):
    """
    `tokens`  = a POS-tagged sentence [(w1, t1), ...]
    `index`   = the index of the token we want to extract features for
    `history` = the previous predicted IOB tags
    """
 
    # init the stemmer
    stemmer = SnowballStemmer('english')
 
    # Pad the sequence with placeholders
    tokens = [('[START2]', '[START2]'), ('[START1]', '[START1]')] + list(tokens) + [('[END1]', '[END1]'), ('[END2]', '[END2]')]
    history = ['[START2]', '[START1]'] + list(history)
 
    # shift the index with 2, to accommodate the padding
    index += 2
 
    word, pos = tokens[index]
    prevword, prevpos = tokens[index - 1]
    prevprevword, prevprevpos = tokens[index - 2]
    nextword, nextpos = tokens[index + 1]
    nextnextword, nextnextpos = tokens[index + 2]
    previob = history[index - 1]
    contains_dash = '-' in word
    contains_dot = '.' in word
    allascii = all([True for c in word if c in string.ascii_lowercase])
 
    allcaps = word == word.capitalize()
    capitalized = word[0] in string.ascii_uppercase
 
    prevallcaps = prevword == prevword.capitalize()
    prevcapitalized = prevword[0] in string.ascii_uppercase
 
    nextallcaps = prevword == prevword.capitalize()
    nextcapitalized = prevword[0] in string.ascii_uppercase
 
    return {
        'word': word,
        'lemma': stemmer.stem(word),
        'pos': pos,
        'all-ascii': allascii,
 
        'next-word': nextword,
        'next-lemma': stemmer.stem(nextword),
        'next-pos': nextpos,
 
        'next-next-word': nextnextword,
        'nextnextpos': nextnextpos,
 
        'prev-word': prevword,
        'prev-lemma': stemmer.stem(prevword),
        'prev-pos': prevpos,
 
        'prev-prev-word': prevprevword,
        'prev-prev-pos': prevprevpos,
 
        'prev-iob': previob,
 
        'contains-dash': contains_dash,
        'contains-dot': contains_dot,
 
        'all-caps': allcaps,
        'capitalized': capitalized,
 
        'prev-all-caps': prevallcaps,
        'prev-capitalized': prevcapitalized,
 
        'next-all-caps': nextallcaps,
        'next-capitalized': nextcapitalized,
    }

Other than that the code is more or less the same.  I leave it as an exercise to the reader to download the data and then process it.  Unfortunately, this can take quite a while so you might want to try this [other blog post by the same author](https://nlpforhackers.io/training-ner-large-dataset/), for faster processing time.

So we've done some cool stuff - built a toy search engine with a number of features.  But we aren't limited to simply searching over data.  One of the coolest things about the two tools we've looked at - part of speech tagging and named entity recognition, is they give us a sense of the syntax and semantics of text.  In a way, textual data is more complex than other forms of data, because it has both of these features.  Together, the two effect human language in subtle, yet important ways.  

For instance:

Let's eat Grandma.

Is very different than:

Let's eat, Grandma.

In the first sentence we are going to eat Grandma (oh no!), in the second sentence we are telling Grandma we are going to go eat.

And it's overall meaning is entirely change by syntax!  Of course, we as humans know that the first sentence is riddiculous, but machines aren't nearly as smart as us when it comes to processing language, so they are likely to take everything you say literally.  But I digress.

Now let's look at another use case for named entity recognition before we move on: Relation Extraction.

## Enter Relation Extraction

While not directly related to information retrieval or anything we've done so far, this concept is so cool I thought I'd give a quick example of it.  The basic idea is to take pairs of named entities and find a bridge word, to combine them into a tuple of the following kind:

(Named entity, bridge word, Named Entity)

So for example:

(Crystler Building, in, New York City)

Let's see a quick programmatic example:

In [3]:
import nltk
import re
IN = re.compile(r'.*\bin\b(?!\b.+ing)')
for doc in nltk.corpus.ieer.parsed_docs('NYT_19980315'):
    for rel in nltk.sem.extract_rels('ORG', 'LOC', doc,
                                     corpus='ieer', pattern = IN):
         print(nltk.sem.rtuple(rel))

[ORG: 'WHYY'] 'in' [LOC: 'Philadelphia']
[ORG: 'McGlashan &AMP; Sarrail'] 'firm in' [LOC: 'San Mateo']
[ORG: 'Freedom Forum'] 'in' [LOC: 'Arlington']
[ORG: 'Brookings Institution'] ', the research group in' [LOC: 'Washington']
[ORG: 'Idealab'] ', a self-described business incubator based in' [LOC: 'Los Angeles']
[ORG: 'Open Text'] ', based in' [LOC: 'Waterloo']
[ORG: 'WGBH'] 'in' [LOC: 'Boston']
[ORG: 'Bastille Opera'] 'in' [LOC: 'Paris']
[ORG: 'Omnicom'] 'in' [LOC: 'New York']
[ORG: 'DDB Needham'] 'in' [LOC: 'New York']
[ORG: 'Kaplan Thaler Group'] 'in' [LOC: 'New York']
[ORG: 'BBDO South'] 'in' [LOC: 'Atlanta']
[ORG: 'Georgia-Pacific'] 'in' [LOC: 'Atlanta']


Using the above framework, we can build up a series of node, edge pairs and build a semantic graph of connections.  Then we can define metrics like:

* number of times entities have a relationship in a given document
* number of times entities have a relationship on average over a series of documents
* number of times entities have a relationship on average per paragraph
* average number of hops needed to build a relationship between two entities
* minimum number of hops between two entities
* 10 most frequently occurring pairs
* 10 least frequently occurring pairs
* most frequent bridge phrase between two entities
* least common bridge phrase between two entities
* The probability entity A is connected to entity B
* The probability entity A is connected to entity B by bridge phrase C
* The probability bridge phrase A is used

Using these measures we can understand basic features about a set of documents and how different entities inter-relate.  Think of it as the fastest possible reading.  Of course, you lose all the nuance, which can be a problem, so it might be useful to recover the paragraphs around the connection for context reasons.

Let's look at an example making use of this notion, first we'll implement each of the metrics as well as write a function for the above code:

In [10]:
import nltk
import re
from collections import defaultdict

class AnalyzeRelationshipsPerDocument:
    def __init__(self, relationships):
        self.relationships = relationships
        self.entities = self.get_entities()
        self.bridge_phrases = self.get_bridge_phrases()
        self.entity_graph = self._build_graph()
        
    def get_entities(self):
        entities = []
        for relationship in self.relationships:
            if relationship[0] not in entities:
                entities.append(relationship[0])
            if relationship[2] not in entities:
                entities.append(relationship[2])
        return entities
    
    def get_bridge_phrases(self):
        bridge_phrases = []
        for relationship in self.relationships:
            if relationship[1] not in bridge_phrases:
                bridge_phrases.append(relationship[1])
        return bridge_phrases
    
    def entity_freq(self):
        frequencies = {}.fromkeys(self.relationships, 0)
        for relationship in self.relationships:
            frequencies[relationship[0]] += 1
            frequencies[relationship[2]] += 1
        return frequencies
    
    def _build_graph(self):
        graph = defaultdict(list)
        for relationship in relationships:
            graph[relationship[0]].append(relationship[2])
            graph[relationship[2]].append(relationship[0])
        return graph
    
IN = re.compile(r'.*\bin\b(?!\b.+ing)')
for doc in nltk.corpus.ieer.parsed_docs('NYT_19980407'):
    for rel in nltk.sem.extract_rels('ORG', 'LOC', doc,
                                     corpus='ieer', pattern = IN):
         print(nltk.sem.rtuple(rel))

[ORG: 'Bayona'] ', which is in the heart of the' [LOC: 'French Quarter']
[ORG: 'CBS Studio Center'] 'in' [LOC: 'Studio City']
[ORG: 'Smithsonian Institution'] 'in' [LOC: 'Washington']
[ORG: 'Metropolitan Museum of Art'] 'in' [LOC: 'New York']
[ORG: 'British Museum'] 'door is not in' [LOC: 'Washington']


In [5]:
nltk.corpus.ieer.fileids()

['APW_19980314',
 'APW_19980424',
 'APW_19980429',
 'NYT_19980315',
 'NYT_19980403',
 'NYT_19980407']